# CM50270 Reinforcement Learning
## Coursework Part 1: Value Iteration

In this exercise, you will implement the value iteration algorithm for three closely related, but different, gridworld environments.

**Total number of marks:** 30 marks.

**What to submit:** Your completed Jupyter notebook (.ipynb file) which should include all of your source code. Please do not change the file name or compress/zip your submission. Please do not include any identifying information on the files you submit. This coursework will be **marked anonymously**.

**Where to submit:** CM50270 Moodle Page.

You are required to **work individually**. You are welcome to discuss ideas with others but you must design your own implementation and write your own code.

**Do not plagiarise**. Plagiarism is a serious academic offence. For details on what plagiarism is and how to avoid it, please visit the following webpage: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

If you are asked to use specific variable names, data-types, function signatures and notebook cells, **please ensure that you follow these instructions**. Not doing so will cause the automarker to reject your work, and will assign you a score of zero for that question. **If the automarker rejects your work because you have not followed the instructions, you may not get any credit for your work**.

Please remember to **save your work regularly**.

Please be sure to **restart the kernel and run your code from start-to-finish** (Kernel → Restart & Run All) before submitting your notebook. Otherwise, you may not be aware that you are using variables in memory that you have deleted.

Your total runtime must be less than **1 minute** on the University's computers in 10W 0.02. Otherwise, you may not get credit for your work.

## Introduction
In this coursework, you will implement the Value Iteration algorithm to compute an optimal policy for three different (but closely related) Markov Decision Processes. For your reference, the pseudo-code for the Value Iteration algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 2018, pp. 83).

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The symbol $r$ represents the immediate reward on transition from state $s$ to the next state $s'$ via action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 300px;" align="left" caption="Figure 1"/>

The three problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this kind of environment from the lectures. The grid squares in the figure are numbered as shown. In all exercises, the following are true: 

**Actions available:** The agent has four possible actions in each grid square. These are *west*, *north*, *south*, and *east*. If the direction of movement is blocked by a wall (for example, if the agent executes action south at grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold (from a neighbouring grid square), the agent collects the gold. Note that, in order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square.

**Hitting the bomb:** On arrival at a grid square that contains a bomb (from a neighbouring grid square), the agent activates the bomb. 

**Terminal states:** The game terminates when all gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.


### Instructions ###
Set parameter $\theta$ to 1 to the power of -10. You can express that as `1e-10` in Python. 

Set all initial state values $V(s)$ to zero.

Do not use discounting (that is, set $\gamma=1$).

Use the following reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. For example, the immediate reward for transitioning into a square with gold (from a neighbouring grid square) is $-1 + 10 = +9$. 

## Exercise 1: Deterministic Environment (5 Marks)

In this exercise, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment.

Compute the optimal policy using Value Iteration. 

Your need to produce two one-dimensional numpy arrays with names `policy` and `v`. Both arrays should have a length of 25, with the element at index $i$ representing grid cell $i$ (see figure above). Both arrays should be accessible in the "solution cell" below!

The array `policy` should be a numpy array of strings that specifies an optimal action at each grid location. Please use the abbreviations "n", "e", "s", and "w" for the four actions. As an example, the value of `policy` at index `0` needs to give `"n"`, if _north_ is an optimal action in cell 0. The policy for a terminal state can be any action. If there are multiple optimal actions from a state, any optimal action will be considered as a correct answer. 

The array `v` should be an array of floats that contains the expected return at each grid square (that is, the state value under the optimal policy).

In [ ]:
import numpy as np
import itertools

In [ ]:
class GridWorld(object):
    
    def __init__(self, nCols=5, nRows=5, random_move_prob=0.2, bomb_loc=(18, ), gold_loc=(23, )):
        
        #Initialise values that define the grid and some other essential parameters
        self.nCols = nCols
        self.nRows = nRows
        if self.nRows<2 or self.nCols<2:
            raise ValueError("Number of columns and rows may not be smaller than 2")
        self.nCells = nCols*nRows
        self.random_move_prob = random_move_prob
        self.bomb = np.array(bomb_loc)
        self.gold = np.array(gold_loc)
         
        #Set reward array
        self.reward = np.zeros(self.nCells)
        self.reward[self.gold] = 10
        self.reward[self.bomb] = -10
        
        #Define possible actions
        self.actions = ["n", "e", "s", "w"]
        self.nActions = len(self.actions)
        
    def getBellmanValues(self, state, action):
        transition_probs = np.zeros(self.nActions)
        rewards = np.zeros_like(transition_probs)
        new_states = np.zeros_like(transition_probs, dtype=int)
        for i, a in enumerate(self.actions):
            new_pos = state
            if a == "s":
                proposed_pos = state - self.nCols
                if proposed_pos>=0:
                    new_pos = proposed_pos
            elif a == "n":
                proposed_pos = state + self.nCols
                if proposed_pos<self.nCells:
                    new_pos = proposed_pos
            elif a == "w":
                proposed_pos = state - 1
                if state%self.nCols != 0:
                    new_pos = proposed_pos
            elif a == "e":
                proposed_pos = state + 1
                if state%self.nCols != self.nCols - 1:
                    new_pos = proposed_pos
            
            new_states[i] = new_pos
            if a==action:
                transition_probs[i]=1 - self.random_move_prob + self.random_move_prob/(self.nActions)
            else:
                transition_probs[i]=self.random_move_prob/(self.nActions)

            rewards[i] = self.reward[new_pos]
            rewards[i] += -1
            
        return transition_probs, rewards, new_states

In [ ]:
def value_iteration(env, gamma, theta=1e-10):
    delta = 1
    n_states = env.nCells
    n_actions = env.nActions
    bomb = env.bomb
    gold = env.gold
    value_array = np.zeros(n_states)
    policy_array = np.array(['n' for i in range(n_states)])
    nCols = env.nCols
    nRows = env.nRows
    index = 0
    while delta > theta:
        delta = 0
        for state in range(n_states):
            if state not in (*gold, *bomb):
                value_current = value_array[state]
                bellman_sums = np.zeros(n_actions)
                
                for action_index in range(n_actions):
                    action = env.actions[action_index]
                    transition_probs, expected_rewards, new_states = env.getBellmanValues(state, action)
                    bellman_sums[action_index] = np.sum(transition_probs*(expected_rewards + gamma*value_array[new_states]))
        
                value_array[state] = np.max(bellman_sums)
                delta = np.max([delta, np.abs(value_current - value_array[state])])
                best_action_index=np.argmax(bellman_sums)
                best_action = env.actions[best_action_index]
                policy_array[state] = best_action
                
    return value_array, policy_array

In [ ]:
# Please write your code for Exercise 1 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE
env = GridWorld(random_move_prob=0) #deterministic gridworld environment
v, policy = value_iteration(env, 1)
print(v)

#### Check the data types of your solution! 
Below, we have provided a number of example tests which allow you to check whether the variables `policy` and `v` (which you should have computed *above* this cell) have the correct data-types. Feel free to re-use these tests to check the data-types for the remaining exercises, and construct your own similar tests for later pieces of coursework.

In [ ]:
# Example arrays with CORRECT data types (but WRONG values!) would be:
# policy = np.array(["n", "w", "s", "w", "e", "n", "w", "s", "w", "e", 
#                    "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", 
#                    "n", "w", "s", "w", "e"])
# v = np.random.rand(25)
# DO NOT UNCOMMENT THE PREVIOUS lines in your submission... otherwise they will overwrite the arrays that you computed!

# Print the values you computed
print("This is your 'policy':")
print(policy)
print("These are your state values 'v':")
print(v)

# Check whether both policy and v are numpy arrays.
import numpy as np
assert(isinstance(policy, np.ndarray))
assert(isinstance(v, np.ndarray))

# Check correct shapes of numpy arrays.
assert(policy.shape == (25, ))
assert(v.shape == (25, ))

# Check whether the numpy arrays have the correct data types.
assert(np.issubdtype(policy.dtype, np.unicode_)) # policy.dtype should be '<U1'
assert(np.issubdtype(v.dtype, np.float64))

# Check whether all policy values are either "n", "w", "s", or "e".
assert(np.all(np.isin(policy, np.array(["n", "w", "s", "e"]))))

In [ ]:
# DO NOT DELETE THIS CELL!
# Your code for Exercise 1 is tested here.


## Exercise 2: Stochastic Environment (10 Marks)

In this exercise, we introduce stochasticity into the environment. Now, the agent is not always able to execute its actions as intended.

With probability 0.8, the agent moves as intended. However, with probability 0.2, it moves in a random direction.

For example, from grid square 0, if the agent tries to move north, with probability 0.8 the action will work as intended. But with probability 0.2, the agent's motor control system will move it in a random direction (including north). So, it will randomly try to move west, east, north or south with probability 0.05 each. Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.
 
Compute the optimal policy using Value Iteration.

Your value iteration method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercise 1.

In [ ]:
# Please write your code for Exercise 2 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE
env_stochastic = GridWorld(random_move_prob=0.2)
v, policy = value_iteration(env_stochastic, 1)

In [ ]:
# DO NOT DELETE THIS CELL!
# Your code for Exercise 2 is tested here.


## Exercise 3: Stochastic Environment with Two Pieces of Gold (15 marks)

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this exercise, we have modified the stochastic environment presented in exercise 2. A second piece of gold has been placed on grid square 12. The terminal state is reached only when _all_ pieces of gold are collected or when the bomb is activated.

Compute the optimal policy for this altered environment using Value Iteration.

Hint: You will need to change your state representation in order to account for the additional piece of gold.

Your method should output two one-dimensional numpy arrays with names `policy` and `v`, as in the previous exercises. These arrays should specify the expected return and an optimal policy at the corresponding grid sqaure **before any pieces of gold are collected or a bomb is activated.** 

In [ ]:
class GridWorld_NGold(object):
    
    def __init__(self, columns=5, rows=5, gold_loc=(23, 12), bomb_loc=(18,), random_step_probability=0.2):
        self.cols = columns
        self.rows = rows
        self.cells = columns * rows
        self.gold = len(gold_loc)
        self.gold_loc = gold_loc
        self.bomb_loc = bomb_loc
        self.rand_step_prob = random_step_probability
        
        self.loc_states = [i for i in range(self.cells)]
        self.gold_states = self.calc_gold_permutations()
        self.states = list(itertools.product(self.loc_states, self.gold_states))
        self.final_states = [state for state in self.states if len(state[1]) == self.gold
                            or state[0] in bomb_loc]
        
        self.actions = ["n", "w", "s", "e"]
        self.nActions = len(self.actions)
        
    def calc_gold_permutations(self):
        n = len(self.gold_loc)
        permutations=[list(itertools.combinations([i for i in range(n)], j)) for j in range(n+1)]
        gold_states = []
        for i,permutation in enumerate(permutations):
            if len(permutation)>=1:
                for gold in permutation:
                    gold_states.append(list(gold))
        return gold_states
    
    def calc_new_location(self, old_location, action):
        new_pos = old_location
        if action == "s":
            proposed_pos = old_location - self.cols
            if proposed_pos>=0:
                new_pos = proposed_pos
        elif action == "n":
            proposed_pos = old_location + self.cols
            if proposed_pos<self.cells:
                new_pos = proposed_pos
        elif action == "w":
            proposed_pos = old_location - 1
            if old_location%self.cols != 0:
                new_pos = proposed_pos
        elif action == "e":
            proposed_pos = old_location + 1
            if old_location%self.cols != self.cols - 1:
                new_pos = proposed_pos  
        return new_pos
        
    def calc_prob(self, old_location, old_gold, new_gold, action, alt_action):
        old_loc = old_location
        prop_loc = self.calc_new_location(old_loc, alt_action)
        new_loc = self.calc_new_location(old_loc, action)
        
        if action == alt_action:
            prob = 1 - self.rand_step_prob + self.rand_step_prob/self.nActions
        else:
            prob = self.rand_step_prob/self.nActions
        
        isGold = new_loc in self.gold_loc
        if not isGold:
            if new_gold != old_gold:
                prob = 0
        else:
            gold_index = self.gold_loc.index(new_loc)
            if gold_index in old_gold:
                if set(old_gold)!=set(new_gold):
                    prob = 0
            
            elif len(new_gold) > len(old_gold)+1:
                prob = 0
            elif set(old_gold +  [gold_index]) != set(new_gold):
                prob = 0
                
        return prob   
    
    def calc_reward(self, old_location, old_gold, action):
        old_loc = old_location
        new_loc = self.calc_new_location(old_loc, action)
        reward = -1
        if new_loc in self.bomb_loc:
            reward -= 10
        elif new_loc in self.gold_loc:
            gold_index = self.gold_loc.index(new_loc)
            if gold_index not in old_gold:
                reward += 10
    
        return reward 
    
    def calc_bellman_values(self, old_location, old_gold, action):
        n = len(self.gold_states)*self.nActions # number of possible new states following action
        new_states = [(None,None) for i in range(n)]
        transit_probs = np.zeros(n)
        rewards = np.zeros(n)
        state_indices = np.zeros(n, dtype=int)
        old_loc = old_location
        index = 0
        
        for alt_action in self.actions:
            alt_loc = self.calc_new_location(old_loc, alt_action)
            for new_gold in self.gold_states:
                reward = self.calc_reward(old_location, old_gold, alt_action)
                rewards[index] = reward
                transit_probs[index] = self.calc_prob(old_loc, old_gold, new_gold, action, alt_action)
                new_states[index]=(alt_loc, new_gold)
                state_indices[index]=self.states.index((alt_loc, new_gold))
                index+=1
                
        return transit_probs, rewards, state_indices       

In [ ]:
def value_iteration_NGold(env, gamma, theta=1e-10):
    delta = 1
    n_loc = env.cells
    n_gold = len(env.gold_states)
    states = env.states
    n_states = len(states)
    n_actions = env.nActions
    value_array = np.zeros(n_states)
    policy_array = ["n" for i in range(n_states)]
    
    while delta > theta:
        delta = 0
        for i, state in enumerate(states):
            loc, gold = state
            if state not in env.final_states:
                value_current = value_array[i]
                bellman_sums = np.zeros(n_actions)
                for action_index in range(n_actions):
                    action = env.actions[action_index]
                    transition_probs, expected_rewards, state_indices = env.calc_bellman_values(loc, gold, action)
                    values = value_array[state_indices]
                    bellman_sums[action_index] = np.sum(transition_probs*(expected_rewards + gamma*values))
                   
                value_array[i] = np.max(bellman_sums)
                delta = np.max([delta, np.abs(value_current - value_array[i])])
                best_action_index=np.argmax(bellman_sums)
                best_action = env.actions[best_action_index]
                policy_array[i] = best_action
                
    return value_array, policy_array

In [ ]:
# Please write your code for Exercise 3 in this cell or in as many cells as you want ABOVE this cell.
# Your code should compute the values of policy and v from scratch when this cell is executed, 
# using the value iteration algorithm. We will mark your coursework by checking the values of 
# the variables policy and v in the hidden test cell further below. Do NOT delete this cell.

# YOUR CODE HERE

env_NGold = GridWorld_NGold(gold_loc=(23,), random_step_probability=0.)
value_array, policy_array = value_iteration_NGold(env_NGold, 1.)
states_0 = [state for state in env_NGold.states if state[1]==[]] #States with no gold collected yet
state_0_indices = [env_NGold.states.index(state) for state in states_0] #Indices of states with no gold collected yet

policy = [policy_array[index] for index in state_0_indices] #policy for states with no collected gold
v = [value_array[index] for index in state_0_indices] #value array for states with no collected gold

In [ ]:
# DO NOT DELETE THIS CELL!
# Your code for Exercise 3 is tested here.
